# WPILib ML Notebook


## Introduction

By using this notebook, you can train a TensorFlow Lite model for use on a Raspberry Pi and Google Coral USB Accelerator. We've designed this process to be as simple as possible. If you find an issue with this notebook, please create a new issue report on our [GitHub page](https://github.com/GrantPerkins/CoralSagemaker), where you downloaded this notebook.

### Training

1. Download the WPILIB dataset as a .tar file [here](https://github.com/GrantPerkins/CoralSagemaker/releases/download/v1/WPILib.tar)
2. Upload your .tar file to a new folder in an Amazon S3 bucket, or a brand new S3 bucket.
3. Create a new SageMaker notebook instance, and open the WPILib notebook.
4. Change estimator.fit() in the last code cell to use your new dataset, by specifying the folder in which the tar is stored.
5. Run the code block.
6. Training should take roughly 10 minutes and cost roughly \\$0.55 if using the GPU instance, or 45 minutes and cost roughly \\$0.45 if using the CPU instance. If you do not change anything in the notebook, other than the S3 location, it should absolutely not take longer than an hour.

## Notebook


In [1]:
%%sh
#!/usr/bin/env bash
docker system prune --force > /dev/null 2>&1

# The name of our algorithm
algorithm_name=wpi-gpu

cd container

chmod -R +x coral/

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email) 2> /dev/null

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} . --no-cache # > /dev/null 2>&1
docker tag ${algorithm_name} ${fullname} > /dev/null 2>&1

docker push ${fullname} #> /dev/null 2>&1

Login Succeeded
Sending build context to Docker daemon  124.9kB
Step 1/18 : FROM tensorflow/tensorflow:1.12.0-rc2-devel-gpu
1.12.0-rc2-devel-gpu: Pulling from tensorflow/tensorflow
18d680d61657: Pulling fs layer
0addb6fece63: Pulling fs layer
78e58219b215: Pulling fs layer
eb6959a66df2: Pulling fs layer
f7221b275362: Pulling fs layer
c7ca04507ec7: Pulling fs layer
d9e715e7068d: Pulling fs layer
baf5b702c331: Pulling fs layer
eb330861cdf2: Pulling fs layer
2441121cf9c3: Pulling fs layer
bb13cab9e8e9: Pulling fs layer
b5457b255e98: Pulling fs layer
39628178caf5: Pulling fs layer
6f161e5b9ca4: Pulling fs layer
861c0316324d: Pulling fs layer
9fa8bcd890df: Pulling fs layer
df5a9509bf55: Pulling fs layer
bab3ba4084cc: Pulling fs layer
1877b28fd83d: Pulling fs layer
1068d290c01c: Pulling fs layer
eb6959a66df2: Waiting
f7221b275362: Waiting
c7ca04507ec7: Waiting
d9e715e7068d: Waiting
baf5b702c331: Waiting
eb330861cdf2: Waiting
2441121cf9c3: Waiting
b5457b255e98: Waiting
bb13cab9e8e9: Waiting
3

This step runs the training instance (default for GPU is a ml.p3.2xlarge and for the default is CPU is an ml.c4.2xlarge), and begins training with the data specified in `fit()`

This section has lots of configurable values
You need to change `estimator.fit(...)`:to be the location of the data used for training. (the bucket you uploaded the .tar to) It should be in the format `"s3://BUCKET-NAME"`


In [2]:
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role


# Uses GPU by default, change to false to use CPU
use_gpu = True

role = get_execution_role()

instance_type = None
algorithm_name = None

import boto3

client = boto3.client('sts')
account = client.get_caller_identity()['Account']

my_session = boto3.session.Session()
region = my_session.region_name

if not use_gpu:
    instance_type = 'ml.c4.2xlarge'
    algorithm_name = 'wpi-cpu'
else:
    instance_type = 'ml.p3.2xlarge'
    algorithm_name = 'wpi-gpu'

# The number of epochs to train to. 1000 is a safe number. With the default instance, it should take 45 minutes.
hyperparameters = {'epochs': 150,
                   'batch_size': 64}

ecr_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account, region, algorithm_name)

# The estimator object, using our notebook, training instance, the ECR image, and the specified training steps
estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type=instance_type,
                      image_name=ecr_image,
                      hyperparameters=hyperparameters)

# Change this bucket if you want to train with your own data. The WPILib bucket contains thousands of high quality labeled images.
# s3://wpilib
estimator.fit("s3://wpilib")


2020-01-02 23:10:08 Starting - Starting the training job...
2020-01-02 23:10:10 Starting - Launching requested ML instances......
2020-01-02 23:11:16 Starting - Preparing the instances for training...
2020-01-02 23:12:09 Downloading - Downloading input data...
2020-01-02 23:12:25 Training - Downloading the training image............
2020-01-02 23:14:29 Training - Training image download completed. Training in progress..
Successfully created the TFRecords: /opt/ml/input/data/training/train.record.
Successfully created the TFRecords: /opt/ml/input/data/training/eval.record.
Records generated.
Hyperparameters parsed.
Beginning training on Docker image
Results of training:
    Checkpoint 100 accuracy: 16.795%
    Checkpoint 150 accuracy: 19.518%

Checkpoint 150 will be converted..
Converting checkpoint to tflite.
Compiling model for Edge TPU

2020-01-02 23:21:28 Uploading - Uploading generated training model
2020-01-02 23:21:54 Completed - Training job completed
Training seconds: 585
Billa